In [1]:
%cd ..
%pip install -e .

/home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
Obtaining file:///home/sandor/dtu/2021-22-spring/advanced_machine_learning/p-reg
  Preparing metadata (setup.py) ... done
  Attempting uninstall: src
    Found existing installation: src 0.1.0
    Uninstalling src-0.1.0:
      Successfully uninstalled src-0.1.0
  Running setup.py develop for src
Note: you may need to restart the kernel to use updated packages.


In [5]:
import random

from src.models.dense import NN0
from src.models.dense import NN1
from src.models.gcn import GCN0

from src.models.train_model import train0
from src.models.train_model import train1
from src.models.train_model import train2
from src.models.train_model import set_masks
from src.models.train_model import random_splits

from src.models.evaluate_model import evaluate0

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.manifold import TSNE

In [13]:
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0].to(device)
mu = 0.058
for A, B in [[5,2], [10,5], [20, 5], [40, 10], [100, 20]]:
    torch.manual_seed(0)
    random.seed(0)
    data = random_splits(data, A, B)
    
    gcn_model = GCN0(num_node_features=dataset.num_node_features,
                    num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train0(gcn_model, data)
    acc = evaluate0(gcn_model, data)

    print(f'A: {A}, B: {B}, mu: {mu}, noreg, Accuracy: {acc:.4f}')

    gcn_model = GCN0(num_node_features=dataset.num_node_features,
                    num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train1(gcn_model, data, mu=mu)
    acc = evaluate0(gcn_model, data)
    print(f'A: {A}, B: {B}, mu: {mu}, reg, Accuracy: {acc:.4f}')
    print('----------------------')


A: 5, B: 2, mu: 0.058, noreg, Accuracy: 0.7503
A: 5, B: 2, mu: 0.058, reg, Accuracy: 0.7484
----------------------
A: 10, B: 5, mu: 0.058, noreg, Accuracy: 0.8033
A: 10, B: 5, mu: 0.058, reg, Accuracy: 0.7956
----------------------
A: 20, B: 5, mu: 0.058, noreg, Accuracy: 0.8038
A: 20, B: 5, mu: 0.058, reg, Accuracy: 0.6668
----------------------
A: 40, B: 10, mu: 0.058, noreg, Accuracy: 0.8346
A: 40, B: 10, mu: 0.058, reg, Accuracy: 0.2583
----------------------
A: 100, B: 20, mu: 0.058, noreg, Accuracy: 0.8319
A: 100, B: 20, mu: 0.058, reg, Accuracy: 0.3742
----------------------


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0].to(device)

for mu in range(10):
    mu = 0.05 + mu*0.001
    torch.manual_seed(1)
    random.seed(1)

    data = random_splits(data, 10, 5)

    gcn_model = GCN0(num_node_features=dataset.num_node_features,
                    num_classes=dataset.num_classes) \
                    .to(device)

    gcn_model = train1(gcn_model, data, mu=mu)
    # gcn_model = train2(gcn_model, data)
    acc = evaluate0(gcn_model, data)
    print(f'mu: {mu}, reg, Accuracy: {acc:.4f}')
    print('--------------------------------------------')

mu: 0.05, reg, Accuracy: 0.7910
--------------------------------------------
mu: 0.051000000000000004, reg, Accuracy: 0.7906
--------------------------------------------
mu: 0.052000000000000005, reg, Accuracy: 0.7922
--------------------------------------------
mu: 0.053000000000000005, reg, Accuracy: 0.7914
--------------------------------------------
mu: 0.054000000000000006, reg, Accuracy: 0.7899
--------------------------------------------
mu: 0.055, reg, Accuracy: 0.7902
--------------------------------------------
mu: 0.056, reg, Accuracy: 0.7918
--------------------------------------------
mu: 0.057, reg, Accuracy: 0.7914
--------------------------------------------
mu: 0.058, reg, Accuracy: 0.7918
--------------------------------------------
mu: 0.059000000000000004, reg, Accuracy: 0.7906
--------------------------------------------
